In [ ]:
# import os
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

#### Helpers

In [ ]:
# stdlib
import time


class TimeoutError(Exception):
    pass


class Timeout:
    def __init__(self, timeout_duration):
        if timeout_duration > 60:
            raise ValueError("Timeout duration cannot exceed 60 seconds.")
        self.timeout_duration = timeout_duration

    def run_with_timeout(self, condition_func, *args, **kwargs):
        start_time = time.time()
        result = None

        while True:
            elapsed_time = time.time() - start_time
            if elapsed_time > self.timeout_duration:
                raise TimeoutError(
                    f"Function execution exceeded {self.timeout_duration} seconds."
                )

            # Check if the condition is met
            try:
                if condition_func():
                    print("Condition met, exiting early.")
                    break
            except Exception as e:
                print(f"Exception in target function: {e}")
                break  # Exit the loop if an exception occurs in the function
            time.sleep(1)

        return result

### Import lib

In [ ]:
# stdlib
import os

environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
environment

In [ ]:
# stdlib

# syft absolute
import syft as sy

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
    reset=True,
)

In [ ]:
high_client = sy.login(
    url="http://localhost:8080", email="info@openmined.org", password="changethis"
)

In [ ]:
high_client.worker_pools

In [ ]:
default_worker_pool = high_client.worker_pools.get_by_name("default-pool")
default_worker_pool

### Scale Worker pool

In [ ]:
# Scale to 1
high_client.api.worker_pool.scale(number=1, pool_name=default_worker_pool.name)

In [ ]:
high_client.api.services.worker_pool[0]

In [ ]:
# Scale up workers
scale_up_result = high_client.api.worker_pool.scale(
    number=5, pool_name=default_worker_pool.name
)
scale_up_result

In [ ]:
assert scale_up_result

In [ ]:
assert high_client.api.services.worker_pool[default_worker_pool.name].max_count == 5

##### Scale down gracefully

In [ ]:
# Scale down workers, this gracefully shutdowns the consumers
scale_down_result = high_client.api.worker_pool.scale(
    number=1, pool_name=default_worker_pool.name
)
scale_down_result

In [ ]:
assert scale_down_result, scale_down_result

In [ ]:
def has_worker_scaled_down_to_one():
    return high_client.api.worker_pool[default_worker_pool.name].max_count == 1


worker_scale_timeout = Timeout(timeout_duration=20)
worker_scale_timeout.run_with_timeout(has_worker_scaled_down_to_one)

In [ ]:
assert high_client.api.worker_pool[default_worker_pool.name].max_count == 1

##### Scale down forcefully

In [ ]:
# First scale up
high_client.api.services.worker_pool.scale(number=5, pool_name=default_worker_pool.name)

In [ ]:
assert high_client.api.services.worker_pool["default-pool"].max_count == 5

In [ ]:
# Forcefully scale down workers, in this case the workers are terminated immediatedly.

In [ ]:
high_client.api.services.worker_pool.scale(
    1, pool_name=default_worker_pool.name, force=True
)

In [ ]:
assert high_client.api.services.worker_pool[default_worker_pool.name].max_count == 1